# Unit testing lab

### Testing cards

Using the `cards.py` module as our 'unit under test', create a `unittest.TestCase` with tests check that

- a new deck has 52 cards
- a new deck does not repeat any cards
- two cards with the same rank & suit compare as the same
- adding invalid ranks/suits raises an appropriate exception

In [1]:
%%file data/test-examples/card-test-lab.py
import unittest

from cards import Hand, Card, Deck

class TestDeck(unittest.TestCase):
    
    def setUp(self):
        self.deck = Deck()
        
    def test52(self):
        self.assertEqual(len(self.deck), 52)
        
    def test_no_dupes(self):
        seen = set()
        for card in self.deck:
            self.assertNotIn(card, seen)
            seen.add(card)
            
    def test_no_dupes2(self):
        self.assertEqual(
            len(set(self.deck)), 
            len(self.deck),
        )
            
class TestCards(unittest.TestCase):
    
    def test_same_card(self):
        c0 = Card(rank='A', suit='clubs')
        c1 = Card(rank='A', suit='clubs')
        self.assertEqual(c0, c1)

    def test_different_card(self):
        c0 = Card(rank='A', suit='clubs')
        c1 = Card(rank='K', suit='clubs')
        self.assertNotEqual(c0, c1)

    def test_invalid_rank(self):
        with self.assertRaises(ValueError):
            Card(rank='B', suit='clubs')
            
    def test_invalid_suit(self):
        with self.assertRaises(ValueError):
            Card(rank='A', suit='clubsoda')
            
    def test_invalid_suit_a(self):
        self.assertRaises(ValueError, Card, rank='A', suit='clubsoda')         
        
    def test_invalid_suit_b(self):
        try:
            Card(rank='A', suit='clubsoda')
        except ValueError:
            return 
        else:
            raise AssertionError('ValueError not raised')

class TestHand(unittest.TestCase):
    
    def test_simple(self):
        hand = Hand([Card(rank='5', suit='spades')])
        self.assertEqual(hand.score(), 5)
        
    def test_soft_17(self):
        hand = Hand([
            Card(rank='A', suit='spades'),
            Card(rank='6', suit='spades'),
        ])
        self.assertEqual(hand.score(), 17)
            
    def test_hard_17(self):
        hand = Hand([
            Card(rank='A', suit='spades'),
            Card(rank='K', suit='spades'),
            Card(rank='6', suit='spades'),
        ])
        self.assertEqual(hand.score(), 17)
        
    def test_really_hard_14(self):
        hand = Hand([
            Card(rank='A', suit='spades'),
            Card(rank='A', suit='clubs'),
            Card(rank='A', suit='hearts'),
            Card(rank='A', suit='diamonds'),
            Card(rank='K', suit='spades')
        ])
        self.assertEqual(hand.score(), 14)
        


Overwriting data/test-examples/card-test-lab.py


In [2]:
%%bash
cd data/test-examples
python -m unittest card-test-lab.py

.............
----------------------------------------------------------------------
Ran 13 tests in 0.001s

OK


### Evaulating coverage

Use the coverage module to see which lines are missing test coverage. Add tests that exercise these lines in cards.py

In [3]:
%%bash
cd data/test-examples
coverage run -m unittest card-test-lab.py
coverage report -m

Name                                                                            Stmts   Miss  Cover   Missing
-------------------------------------------------------------------------------------------------------------
/home/rick446/.virtualenvs/classes/lib/python3.8/site-packages/_virtualenv.py      81     80     1%   4-54, 57-130
card-test-lab.py                                                                   50      1    98%   54
cards.py                                                                           44      6    86%   19, 34, 43, 46, 49, 67
-------------------------------------------------------------------------------------------------------------
TOTAL                                                                             175     87    50%


.............
----------------------------------------------------------------------
Ran 13 tests in 0.002s

OK


In [4]:
%%bash
cd data/test-examples
coverage erase
coverage run -m unittest card-test-lab.py
coverage annotate

.............
----------------------------------------------------------------------
Ran 13 tests in 0.002s

OK


In [5]:
!cat data/test-examples/cards.py,cover

> ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
> suits = 'spades hearts clubs diamonds'.split()
  
> class Card:    
>     def __init__(self, rank, suit):
>         if rank not in ranks:
-             raise ValueError('invalid rank')  # pragma: no cover
>         if suit not in suits:
>             raise ValueError('invalid suit')
>         self.rank, self.suit = rank, suit
          
>     def __eq__(self, other):   
>         return self.rank == other.rank and self.suit == other.suit
      
>     def __hash__(self):
>         return hash((self.rank, self.suit))
      
>     def __repr__(self):
!         return f'{self.rank} {self.suit}'
      
      
> class CardStack:
      
>     def __init__(self, cards):
>         self.cards = list(cards)
          
>     def __len__(self):
>         return len(self.cards)
      
>     def __getitem__(self, i):
>         return self.cards[i]
      
>     def __repr__(self):
!         return ' '.join(repr(c) for c in self)
      
      
> class De

In [6]:
%%file data/test-examples/card-test-lab2.py
import unittest
import random

from cards import Hand, Card, Deck

class TestDeck(unittest.TestCase):
    
    def setUp(self):
        self.deck = Deck()
        
    def test52(self):
        self.assertEqual(len(self.deck), 52)
        
    def test_no_dupes(self):
        seen = set()
        for card in self.deck:
            self.assertNotIn(card, seen)
            seen.add(card)
            
    def test_repr_string(self):
        "smoke test"
        self.assertIsInstance(repr(self.deck), str)
        
    def test_shuffle(self):
        deck2 = Deck()
        random.shuffle(deck2)
        self.assertNotEqual(self.deck.cards, deck2.cards)
        
    def test_deal_5(self):
        hand = self.deck.deal(5)
        self.assertEqual(len(hand), 5)
        
    def test_draw(self):
        hand = self.deck.deal(5)
        self.deck.draw(hand)
        self.assertEqual(len(hand), 6)
            
class TestCards(unittest.TestCase):
    
    def test_repr_string(self):
        self.assertIsInstance(repr(Card(rank='A', suit='clubs')), str)
            
    def test_same_card(self):
        c0 = Card(rank='A', suit='clubs')
        c1 = Card(rank='A', suit='clubs')
        self.assertEqual(c0, c1)
        
    def test_invalid_rank(self):
        with self.assertRaises(ValueError):
            Card(rank='B', suit='clubs')
            
    def test_invalid_suit(self):
        with self.assertRaises(ValueError):
            Card(rank='A', suit='clubsoda')
            

class TestHand(unittest.TestCase):
    
    def test_simple(self):
        hand = Hand([Card(rank='5', suit='spades')])
        self.assertEqual(hand.score(), 5)
        
    def test_soft_17(self):
        hand = Hand([
            Card(rank='A', suit='spades'),
            Card(rank='6', suit='spades'),
        ])
        self.assertEqual(hand.score(), 17)
            
    def test_hard_17(self):
        hand = Hand([
            Card(rank='A', suit='spades'),
            Card(rank='K', suit='spades'),
            Card(rank='6', suit='spades'),
        ])
        self.assertEqual(hand.score(), 17)
        
    def test_really_hard_14(self):
        hand = Hand([
            Card(rank='A', suit='spades'),
            Card(rank='A', suit='clubs'),
            Card(rank='A', suit='hearts'),
            Card(rank='A', suit='diamonds'),
            Card(rank='K', suit='spades')
        ])
        self.assertEqual(hand.score(), 14)
        


Overwriting data/test-examples/card-test-lab2.py


In [7]:
%%bash
cd data/test-examples
coverage erase
coverage run -m unittest card-test-lab2.py
coverage report -m

Name                                                                            Stmts   Miss  Cover   Missing
-------------------------------------------------------------------------------------------------------------
/home/rick446/.virtualenvs/classes/lib/python3.8/site-packages/_virtualenv.py      81     80     1%   4-54, 57-130
card-test-lab2.py                                                                  52      0   100%
cards.py                                                                           44      0   100%
-------------------------------------------------------------------------------------------------------------
TOTAL                                                                             177     80    55%


..............
----------------------------------------------------------------------
Ran 14 tests in 0.003s

OK


In [ ]:
if condition:
    something()
something_else()

In [8]:
%%bash
cd data/test-examples
coverage erase
coverage run --branch -m unittest card-test-lab2.py
coverage report -m

Name                                                                            Stmts   Miss Branch BrPart  Cover   Missing
---------------------------------------------------------------------------------------------------------------------------
/home/rick446/.virtualenvs/classes/lib/python3.8/site-packages/_virtualenv.py      81     80     26      1     2%   4-54, 55->57, 57-130
card-test-lab2.py                                                                  52      0      2      0   100%
cards.py                                                                           44      0     16      0   100%
---------------------------------------------------------------------------------------------------------------------------
TOTAL                                                                             177     80     44      1    52%


..............
----------------------------------------------------------------------
Ran 14 tests in 0.003s

OK
